In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM


In [48]:
# Read your dataset
weather_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_csv_files/weather_dimension.csv')
county_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_csv_files/County_Information.csv')
air_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_csv_files/air_quality_updated.csv', low_memory=False)
fact_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_csv_files/wellness_fact.csv')

mdata = fact_data.merge(weather_data, left_on='weather_key', right_on='weather_id', how='left')
mdata = mdata.merge(county_data, left_on='county_key', right_on='County_Key', how='left')
mdata = mdata.merge(air_data, left_on='air_key', right_on='index', how='left')

print(mdata.shape)

# Select relevant columns
selected_columns = ['average_temperature', 'exceptional_data_count PM2.5 - Local Conditions quarterly means of daily means Micrograms/cubic meter (LC)', 'ypll', 'uninsured']
mdata = mdata[selected_columns]

# Handle missing values
mdata = mdata.dropna()

mdata


(6251, 160)


,average_temperature,exceptional_data_count PM2.5 - Local Conditions quarterly means of daily means Micrograms/cubic meter (LC),ypll,uninsured
0,66.06,0.0,8221.7,20.90
1,67.73,0.0,8321.8,20.90
2,68.71,0.0,8091.8,16.40
3,66.68,0.0,7770.2,19.10
4,65.77,0.0,7770.2,16.58
...,...,...,...,...
6246,45.63,0.0,5307.8,21.53
6247,43.88,0.0,5329.5,19.01
6248,42.48,0.0,8703.6,17.72
6249,45.47,0.0,8207.7,19.78


In [49]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(mdata)


In [50]:
# Initialize the one-class SVM model
model = OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

# Train the model
model.fit(data_scaled)

# Predict the outliers
outliers = model.predict(data_scaled)

### Debugging ###
ctr=0
for i in outliers:
  if i != 1:
    ctr += 1
print(ctr)
print(outliers.shape)
###############


375
(3759,)


In [51]:
# Find the indices of the outliers
outlier_indices = np.where(outliers == -1)

# Extract outlier data
outlier_data = mdata.iloc[outlier_indices]

# print(outlier_data)

# Analyze outlier data
outlier_summary = outlier_data.describe()

outlier_summary

,average_temperature,exceptional_data_count PM2.5 - Local Conditions quarterly means of daily means Micrograms/cubic meter (LC),ypll,uninsured
count,375.000000,375.000000,375.000000,375.000000
mean,54.877733,122.450667,8414.601333,15.482853
std,12.118797,326.445092,3769.004802,8.478443
min,35.220000,0.000000,3308.200000,2.260000
25%,44.365000,0.000000,5131.850000,7.460000
50%,51.450000,0.000000,7364.900000,14.880000
75%,67.225000,0.000000,10921.800000,22.650000
max,75.420000,2736.000000,22436.400000,39.100000
